In [1]:
import pandas as pd
import numpy as np

In [9]:
salary_data = pd.read_excel("salaire.xls", skiprows=5)
immigration_data = pd.read_excel("immigration.xls", skiprows=10)
population_data = pd.read_excel("population.xls", skiprows=5)
education_data = pd.read_excel("education.xls", skiprows=5)
logement_data = pd.read_excel("logement.xls", skiprows=5)

### First, as usual, we convert unicode to string.

In [12]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode)
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [13]:
salary_data = data_to_string(salary_data)
immigration_data = data_to_string(immigration_data)
population_data = data_to_string(population_data)
education_data = data_to_string(education_data)
logement_data = data_to_string(logement_data)

### Now we will rename the column headers so that we have at least 1 common column for all the data frames. 

In [14]:
def rename_column(data): 
    new_data = data.copy()
    col_title = new_data.columns.tolist()
    for x in xrange(len(col_title)):
        if col_title[x] == 'CODGEO':
            col_title[x] = 'Code Insee' # this will be the pivot column for merging
        if col_title[x] == 'LIBGEO':
            col_title[x] = 'Libelle de la commune'
    new_data.columns = col_title
    return new_data

In [16]:
salary_data = rename_column(salary_data)
immigration_data = rename_column(immigration_data)
population_data = rename_column(population_data)
education_data = rename_column(education_data)
logement_data = rename_column(logement_data)

### We still have problem with departemental code of Corse because they are not in numerical form (2A... or 2B...).

In [17]:
def to_digit(string):
    new_string = list(string)[:]
    for x in xrange(len(new_string)): 
        if not new_string[x].isdigit():
            new_string[x] = '0'
    return int(''.join(new_string))

In [18]:
def replace_insee_code(data): #replace 2AXXX or 2BXXX by 20XXX 
        new_data = data.copy()
        insee_list = new_data['Code Insee'].tolist() 
        new_insee_list = [int(code) if code.isdigit() else to_digit(code) for code in insee_list]
        new_data['Code Insee'] = new_insee_list
        return new_data

In [19]:
salary_data = replace_insee_code(salary_data)
immigration_data = replace_insee_code(immigration_data)
population_data = replace_insee_code(population_data)
education_data = replace_insee_code(education_data)
logement_data = replace_insee_code(logement_data)

### There is still another problem: all the numbers in our excel sheet are read in float format. We need to convert them to int for the immigration and population data because they contain quantity-format data.

In [20]:
def float_to_int(data):
    new_data = data.copy()    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == np.float64: #check the first not-NaN value
            new_data[col] = new_data[col].round()
    return new_data

In [23]:
immigration_data = float_to_int(immigration_data)
population_data = float_to_int(population_data)
education_data = float_to_int(education_data)
logement_data = float_to_int(logement_data )

### In immigration data, we will add a column which shows us the percentage of adult immigrants  in each town.

In [25]:
def add_total_immigrant_variable(data): 
    new_data = data.copy()
    variable_list = []
    sum_list = []
    for variable in data.columns.tolist(): 
        if 'IMMI' in variable and 'AGE400' not in variable: 
            sum_list.append(variable)
            if 'IMMI1' in variable:
                variable_list.append(variable)
    new_data['% immigrant'] = (new_data[variable_list].sum(axis=1)/ new_data[sum_list].sum(axis=1)) * 100
    col = new_data.columns.tolist()
    col = col[:2] + col[-1:] + col[2:-1]
    new_data = new_data[col]
    return new_data

In [26]:
immigration_data = add_total_immigrant_variable(immigration_data)

In [27]:
immigration_data.head()

,Code Insee,Libelle de la commune,% immigrant,AGE400_IMMI1_SEXE1,AGE400_IMMI1_SEXE2,AGE400_IMMI2_SEXE1,AGE400_IMMI2_SEXE2,AGE415_IMMI1_SEXE1,AGE415_IMMI1_SEXE2,AGE415_IMMI2_SEXE1,AGE415_IMMI2_SEXE2,AGE425_IMMI1_SEXE1,AGE425_IMMI1_SEXE2,AGE425_IMMI2_SEXE1,AGE425_IMMI2_SEXE2,AGE455_IMMI1_SEXE1,AGE455_IMMI1_SEXE2,AGE455_IMMI2_SEXE1,AGE455_IMMI2_SEXE2
0,1001,L'Abergement-Clemenciat,4.173623,1,3,92,83,1,0,34,32,7,9,154,137,5,3,101,116
1,1002,L'Abergement-de-Varey,2.105263,0,1,25,18,1,0,6,11,2,1,52,47,0,0,38,32
2,1004,Amberieu-en-Bugey,12.048946,57,53,1386,1542,73,61,802,872,442,403,2447,2386,194,176,1454,1886
3,1005,Amberieux-en-Dombes,4.528012,0,0,176,163,0,2,108,101,18,19,334,324,11,9,182,195
4,1006,Ambleon,8.000000,2,1,5,4,0,0,8,3,2,3,23,19,0,3,18,21


### In education data, we convert the data to percentage. 

In [28]:
def education_level_in_percentage(data):
    new_data = data.copy()
    variable_list = ['Code Insee', 'Libelle de la commune']
    for variable in new_data.columns.tolist(): 
        if '_NSCOL15P_' in variable:
            variable_list.append(variable)
    for chosen_variable in variable_list[2:]:
        new_data[chosen_variable] = (new_data[chosen_variable] / new_data['P12_NSCOL15P']) * 100
        #new_data[chosen_variable] = new_data['P12_NSCOL15P'] - new_data[variable_list].sum(axis=1)
    return new_data[variable_list]

In [29]:
education_data = education_level_in_percentage(education_data)

In [30]:
education_data.head()

,Code Insee,Libelle de la commune,P12_NSCOL15P_DIPL0,P12_NSCOL15P_CEP,P12_NSCOL15P_BEPC,P12_NSCOL15P_CAPBEP,P12_NSCOL15P_BAC,P12_NSCOL15P_BACP2,P12_NSCOL15P_SUP
0,1001,L'Abergement-Clemenciat,14.000000,13.454545,5.818182,25.272727,16.909091,15.272727,9.454545
1,1002,L'Abergement-de-Varey,8.888889,6.666667,3.333333,22.222222,18.333333,21.111111,18.888889
2,1004,Amberieu-en-Bugey,17.637173,9.362333,5.783490,24.824518,19.110232,13.109244,10.173010
3,1005,Amberieux-en-Dombes,16.016949,12.033898,6.186441,29.491525,16.016949,11.610169,8.644068
4,1006,Ambleon,15.555556,7.777778,8.888889,32.222222,22.222222,6.666667,7.777778


### In population data, we will transform the social status columns from quantity to percentage too.

In [31]:
def social_status_percentage(data): 
    new_data = data.copy()
    status_list = ['C12_POP15P_CS1', 'C12_POP15P_CS2','C12_POP15P_CS3','C12_POP15P_CS4','C12_POP15P_CS5',
                   'C12_POP15P_CS6','C12_POP15P_CS7','C12_POP15P_CS8']
    adult_population = new_data['C12_POP15P']
    for col in status_list:
        new_data['% '+ col] = (new_data[col] / adult_population)*100
    return new_data

In [32]:
population_data = social_status_percentage(population_data)

In [33]:
population_data.head()

,Code Insee,REG,DEP,Libelle de la commune,P12_POP,P12_POP0014,P12_POP1529,P12_POP3044,P12_POP4559,P12_POP6074,...,C12_POP55P_CS7,C12_POP55P_CS8,% C12_POP15P_CS1,% C12_POP15P_CS2,% C12_POP15P_CS3,% C12_POP15P_CS4,% C12_POP15P_CS5,% C12_POP15P_CS6,% C12_POP15P_CS7,% C12_POP15P_CS8
0,1001,82,01,L'Abergement-Clemenciat,777,178,97,165,151,128,...,163,8,2.588997,0.647249,12.944984,14.239482,18.770227,11.650485,27.022654,12.297735
1,1002,82,01,L'Abergement-de-Varey,235,44,35,60,38,36,...,64,0,0.000000,2.116402,10.582011,27.513228,4.232804,16.931217,33.862434,4.232804
2,1004,82,01,Amberieu-en-Bugey,14233,3038,2904,2722,2655,1737,...,2816,251,0.000000,2.818540,6.451324,15.595920,17.143880,17.197566,25.590551,15.211167
3,1005,82,01,Amberieux-en-Dombes,1642,338,301,348,359,203,...,321,33,0.000000,5.611068,8.147579,17.524981,18.139892,14.681015,25.288240,10.607225
4,1006,82,01,Ambleon,110,12,18,24,27,20,...,35,0,0.000000,0.000000,4.651163,18.604651,18.604651,9.302326,40.697674,9.302326


### In logement data, we compute the percentage of house proprietor

In [36]:
def proprietor_percentage(data):
    new_data = data.copy()
    new_data['P12_NPER_RP_PROP'] = ((new_data['P12_NPER_RP_PROP'])/new_data['P12_NPER_RP'])*100
    return new_data

In [39]:
logement_data = proprietor_percentage(logement_data)

### Now we try to merge all the data frames

In [40]:
right_party_vote = pd.read_excel("right_party_vote.xlsx")
left_party_vote = pd.read_excel("left_party_vote.xlsx")
er_party_vote = pd.read_excel("er_party_vote.xlsx")

In [42]:
immigration_salary = pd.merge(immigration_data, salary_data, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [43]:
immigration_salary_population = pd.merge(population_data, immigration_salary, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [44]:
immigration_salary_population_education = pd.merge(education_data, immigration_salary_population, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [45]:
im_sal_pop_edu_log = pd.merge(logement_data, immigration_salary_population_education, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [46]:
full_data_right = pd.merge(right_party_vote, im_sal_pop_edu_log, on = 'Code Insee', how = 'left')
full_data_left = pd.merge(left_party_vote, im_sal_pop_edu_log, on = 'Code Insee', how = 'left')
full_data_er = pd.merge(er_party_vote, im_sal_pop_edu_log, on = 'Code Insee', how = 'left')

In [47]:
full_data_right.head()

,Code Insee,Voix,% Voix/Ins,% Voix/Exp,% Abs/Ins,REG_x,DEP_x,Libelle de la commune,P12_LOG,P12_RP,...,SNHMHO12,SNHM1812,SNHM2612,SNHM5012,SNHMF1812,SNHMF2612,SNHMF5012,SNHMH1812,SNHMH2612,SNHMH5012
0,1001,107,18.32,33.44,44.69,82,01,L'Abergement-Clemenciat,335,301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1002,32,14.28,25.20,41.52,82,01,L'Abergement-de-Varey,160,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1004,1096,13.45,29.76,53.34,82,01,Amberieu-en-Bugey,6629,6012,...,10.945976,9.724445,13.042454,15.461354,9.049192,11.33007,12.313855,10.194692,14.088531,17.557177
3,1005,168,14.38,31.22,52.23,82,01,Amberieux-en-Dombes,647,615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1006,22,20.56,42.31,49.53,82,01,Ambleon,70,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Export the full data frame

In [50]:
writer = pd.ExcelWriter('full_data_right.xlsx')
full_data_right.to_excel(writer,'Sheet1')
writer.save()

In [51]:
writer = pd.ExcelWriter('full_data_left.xlsx')
full_data_left.to_excel(writer,'Sheet1')
writer.save()

In [52]:
writer = pd.ExcelWriter('full_data_er.xlsx')
full_data_er.to_excel(writer,'Sheet1')
writer.save()